## Gradient Boosted Trees

This is an implementation of Gradient Boosted Trees for continuous datasets

First we look at importing the required datasets, then defining helper functions for developing a Regression Tree, then relevant classes defining the Regression Tree implementation. After this, the performance of the Regression Tree is compared with Sklearn's DecisionTreeRegressor, showing that for multiple min_samples_split both implementations show very close mse values with test data. 

Then we implement a Gradient Boosted Regression Tree with 5 Trees, and using Mango, a Python library used to find hyperparameters for expensive models, we look at some different implementations using Mango. The main regularisation technique used for our tree is the minimum number of samples split in each node

In [ ]:
!pip3 install numpy pandas scikit-learn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)
df = data.frame
df.columns = range(len(df.columns))
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)
y_train = pd.DataFrame(y_train, columns=[y.name])
y_test = pd.DataFrame(y_test, columns=[y.name])


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# X_train = pd.read_csv("train_x.csv", header=None)
# y_train = pd.read_csv("train_y.csv", header=None)
# X_test = pd.read_csv("test_x.csv", header=None)
# y_test = pd.read_csv("test_y.csv", header=None)

Index([8], dtype='int64')
(14448, 8) (6192, 8) (14448, 1) (6192, 1)


### Helper functions

Here, we define helper functions for fitting the Regression Tree on some training dataset $X$ and $y$. 


In [115]:

# The formula for mse can be rewritten, as the ground_truth is the mean of y values.
# This function uses the sum of y values, the sum of squared y values, and the number of y values to update mse in O(1)
def update_mse(y_sum, y_squared_sum, n):
    return ((y_squared_sum/n) - (y_sum / n)**2)
    # return ((y_squared_sum/n) - (mean)**2)

# Function to find the best split given a dataset, in which the mse of both splits are minimum
def best_mse_split(df):
    n, features = df.iloc[:, :-1].shape
    min_mse = np.inf
    best_feature = 0
    best_value = 0
    best_left_split = []
    best_right_split = []

    # iterating over every feature (feature column in our dataframe)
    for f in range(features):
        # we sort the dataframe by the current feature -> O(n log n) where n is the number of examples
        df_sorted = df.sort_values(by=f, inplace=False)
        y = df_sorted.iloc[:,-1]
        total_y_sum = y.sum()
        total_y_squared_sum = (y**2).sum()
        left_split_y_sum = 0
        left_split_y_squared_sum = 0
        # iterating over every feature vector/ example in the dataset
        for i in range(n-1):  
            # this is O(1), instead of recalculating the mse for the dataset, we can incrementally add an example to the left split and remove it from the right
            # Since calculating mse with the traditional formula takes O(n), by sorting the current feature, we can incrementally control the size of each split
            # and calculate mse of each split in constant time
            left_split_y_sum += y.iloc[i]
            left_split_y_squared_sum += (y.iloc[i])**2
            right_split_y_sum = total_y_sum - left_split_y_sum
            right_split_y_squared_sum = total_y_squared_sum - left_split_y_squared_sum
            left_n = i + 1
            right_n = n - left_n
            
            # we skip duplicate values, if our current dataframe only contains duplicate feature vectors, the algorithm will try to split the dataframe into two, which should be avoided
            if df_sorted.iloc[i,f] == df_sorted.iloc[i+1,f]:
                continue

            left_mse = update_mse(left_split_y_sum, left_split_y_squared_sum, left_n)
            right_mse = update_mse(right_split_y_sum, right_split_y_squared_sum, right_n)
            split_mse = ((left_n*left_mse)+(right_n*right_mse))/n

            # finding current best split
            if split_mse < min_mse:
                min_mse = split_mse
                best_feature = f
                best_value = df_sorted.iloc[i, f]
                # finds the indices where the left split and right split are in the original unsorted dataframe respectively
                best_left_split = np.where(df.iloc[:, f] <= best_value)[0]
                best_right_split = np.where(df.iloc[:, f] > best_value)[0]
    
    return min_mse, best_feature, best_value, best_left_split, best_right_split
            
        

In [123]:
# Recursive function used to build the tree 
def build(df, min_samples):
    n = df.shape[0]
    # stopping condition -> min_samples
    # if the size of the current dataframe in this call is too small we return the mean of the ground truth values
    # becomes a leaf node in our tree
    if n < min_samples:
        return np.mean(df.iloc[:,-1])

    # find the best mse split for the current dataset
    mse, feature, value, left_split, right_split = best_mse_split(df)
    
    # no split could be found, make it a leaf node
    # no non-empty split could be found -> remember if we had duplicate feature vectors for our entire dataset, no split could be found
    # as the mse is never updated, so we return the mean of ground truth values
    if mse == np.inf:
        return np.mean(df.iloc[:,-1])

    # recursively build left and right child of current node
    # using the left and right split respectively
    left_child = build(df.iloc[left_split], min_samples)
    right_child = build(df.iloc[right_split], min_samples)

    return TreeNode(mse, feature, value, left_child, right_child)

# helper function to navigate tree when predicting a new feature vector
def navigate_tree(tree, unseen):
    # leaf node
    if not isinstance(tree,TreeNode):
        return tree

    feature = tree.feature
    value = tree.value

    # recursively call navigate tree
    if unseen[feature] <= value:
        return navigate_tree(tree.left, unseen)
    return navigate_tree(tree.right, unseen)
    
        

### Regression Tree

Below we define the classes for a Tree and the custom RegressionTree class, in which we then compare the performance between the RegressionTree class and Scikit's DecisionTreeRegressor, using mse as the main metric.

In [124]:
# Defining custom Tree class, where each node stores the mse and pointers to children
class TreeNode:

    def __init__(self, mse, feature, value, left, right):
        self.mse = mse
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right

# RegressionTree class, our main Regression Tree implementation
class RegressionTree:

    # storing hyperparameter 
    def __init__(self, min_samples_split):
        self.tree = None
        self.min_samples_split = min_samples_split
        

    # we must concatenate X and y into one dataset for the build function, so we do not have to do that everytime we want to calculate the best mse split
    def fit_tree(self, X, y):
        y = y.rename(columns={0: X.shape[1]})
        train = pd.concat([X,y], axis=1)
        # call recursive build function to create Regression Tree
        self.tree = build(train, self.min_samples_split)

    # predicting y_values for each feature vector in X_test
    def predict(self, X):
        y = []
        for c, r in X.iterrows():
            y.append(navigate_tree(self.tree, r))
        return np.array(y)
        

In [125]:
# example usage 
import time
time_start = time.time()
rt = RegressionTree(100)
rt.fit_tree(X_train, y_train)
time_end = time.time()
print(time_end-time_start)

39.498220682144165


In [126]:
# Running time -> a few minutes
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

# We create a RegressionTree and DecisionTreeRegressor, using a variety of min_samples_split values (the only hyperparameter for the RegressionTree)
# we then fit the trees on X_train and y_train and allow them to make predictions on X_test
# we calculate the mse between predictions and y_test, and append them to an array.
rt_mse = []
scikit_rt_mse = []
for i in range(50,501,50):
    rt = RegressionTree(i)
    rt.fit_tree(X_train, y_train)

    scikit_rt = DecisionTreeRegressor(criterion='squared_error', min_samples_split = i)
    scikit_rt.fit(X_train, y_train)

    rt_prediction = rt.predict(X_test)
    rt_mse.append(mean_squared_error(y_test, rt_prediction))
    scikit_prediction = scikit_rt.predict(X_test)
    scikit_rt_mse.append(mean_squared_error(y_test, scikit_prediction))

# Just by printing the arrays, we see the results are very similiar for min_samples_split ranging from 50 to 500
print(f"My Regression Tree {rt_mse}")
print(f"Sklearns regression tree {scikit_rt_mse}")
# we can go a step further and calculate mse between the two arrays, as mse is just calculating difference in results
final_mse = mean_squared_error(rt_mse, scikit_rt_mse)
print("")
print(f"MSE difference between sklearn's Decision Regressor and my Regression Tree: {final_mse}")

# extremely small, meaning my implementation is likely correct
    

My Regression Tree [0.3770890818150133, 0.36488124820066653, 0.367726984309839, 0.386957383765939, 0.403014212850536, 0.41172716753812144, 0.4164127326689568, 0.4328280747548593, 0.4457439165854728, 0.4533684941844094]
Sklearns regression tree [0.36915327689111344, 0.3608922430135572, 0.36442710197187267, 0.38450484025752985, 0.40136222742026934, 0.4100466429725798, 0.4146757834851642, 0.4315985411513649, 0.44544823790228105, 0.4528911646969541]

MSE difference between sklearn's Decision Regressor and my Regression Tree: 1.0619058862773123e-05


### Ensemble of Regression Trees

Next we will look at an Ensemble of Regression Trees implementation which uses the RegressionTree class:

To fit the tree, we initially set the residuals to just be the target values $y$ (of the training set). We call a RegressionTree with min_samples_split[$i$], where min_samples_split is an array in which each index represents the min_samples_split hyperparameter of each tree $T_i$.

We fit the first tree $T_0$ with $X$_train and the residuals which are just the target values $y$, and then modify the total predictions (currently an array of 0's)s uch that we add the predictions of $T_0$ on $y$. In subsequent iterations, $T_i$ is trained with the residuals, which is the difference between the target values $y$ and the current total predictions $y_pred$, and we of course append the constructed trees to an array.

When we want to predict a $y$-value, we just feed each $T_i$ the test data $X$, and add those predictions to a final value $y$

In [136]:
# RegressionTreeEnsemble class
class RegressionTreeEnsemble:

    # We store n -> number of trees to be implemented and min_samples_split, an array representing the min_samples_split values for n trees
    def __init__(self, n, min_samples_split):
        self.n = n
        self.min_samples_split = min_samples_split
        self.trees = []

    # fit function explained above
    def fit_tree(self, X, y):
        if not self.n <= 0:
            y_pred = pd.DataFrame(0, index=y.index, columns=y.columns)
            for i in range(self.n):
                residuals = y - y_pred
                tree_n = RegressionTree(self.min_samples_split[i])
                tree_n.fit_tree(X, residuals)
                y_pred += pd.DataFrame(tree_n.predict(X), index=y.index, columns = y.columns)         
                self.trees.append(tree_n)

    # predict function explained above
    def predict(self, X):
        if not self.trees:
            print("test")
            return np.array([])

        # first declare y as a matrix of 0's, since we add our predictions onto it from each tree
        y = np.full(X.shape[0], 0, dtype=np.float64)
        print(y)
        for tree in self.trees:
            print(tree.predict(X))
            y += tree.predict(X)
        return y


In [148]:
# example usage -> takes a few minutes
min_samples = [500, 400, 300, 200, 100]
rte = RegressionTreeEnsemble(5, min_samples)
rte.fit_tree(X_train, y_train)
rte_predictions = rte.predict(X_test)
# print(rte_predictions)
print(f"MSE produced by an Ensemble Regression Tree for min_samples_split = {min_samples}: {mean_squared_error(y_test, rte_predictions)}")

[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[-0.3654388   0.32357427  0.41958241 ... -0.27450879  0.09375542
 -0.3654388 ]
[ 0.06946423  0.37233534  0.11039601 ... -0.08507303 -0.14142553
  0.03686439]
[ 0.1882578  -0.11478428 -0.35998773 ...  0.14316987 -0.07072568
  0.1882578 ]
[ 0.12156484 -0.19243007 -0.57086568 ...  0.18968323 -0.0369005
  0.12156484]
MSE produced by an Ensemble Regression Tree for min_samples_split = [500, 400, 300, 200, 100]: 0.34227737631410904


### Hyperparameter Fine-tuning

To achieve the best performance out of the Ensemble Regression Tree on our dataset, fine-tuning of the hyperparameter min_samples_split is required. For the RegressionTree implementation, it may be possible to iterate over a range of min_samples_split value and take the one that yields the lowest MSE, but due to how expensive it is to fit an EnsembleRegressionTree (taking a few minutes each) and the fact that each tree can have different min_samples_split values, careful fine-tuning is required to find a good performing set of min_samples_split parameters whilst also not taking days or weeks to find them. To do this, some hyperparameter fine tuning libraries were used:

#### Skopt

Also known as Scikit Optimize, it has a function gp_minimize, which takes an objective function (which we define in our code) and a set of potential parameters, which for the Ensemble Regression Tree is a range from 2 to 500 for each tree in the Ensemble. It uses Bayesian Optimisation based on a Gaussian Process Model, which first samples the hyperparameter space by evaluating the objective function at random points, and then fits a probabilistic model to predict the objective function in the hyperparameter space. Iteratively, it selects a point of interest and evaluates the objective function at that point, and updates its model to predict where the global minima of the function is. 

This is used with initially 30 calls to the objective function, which took around 4000 seconds to complete, and did not show promising results.

#### Mango

Mango is a parallel hyperparameter tuning library that is capable of fine-tuning in parallel and also uses search techniques based on Bayesian Optimisation. This library is in use by ARM with its main feature being parallelisation. 

This library gave much more promising results than Skopt, however running in parallel does take longer, as we do not split the workload between cores, we let each thread find a minima in parallel, so the number of calls to the objective function is the same for each core as it is for one core.



In [143]:
%pip install arm-mango scikit-optimize

Defaulting to user installation because normal site-packages is not writeable
  Using cached arm_mango-1.4.3-py3-none-any.whl (29 kB)
     |████████████████████████████████| 107 kB 6.7 MB/s eta 0:00:01
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     |████████████████████████████████| 172 kB 11.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [144]:
# simple objective function to be optimised
def obj_func_skopt(min_samples_split):
    # uncomment line below if does not work
    # min_samples_split = [p0, p1, p2, p3, p4]
    model = RegressionTreeEnsemble(5, min_samples_split)
    model.fit_tree(X_train, y_train)
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_true = y_test, y_pred = predictions)
    return mse

In [ ]:
# Skopt implementation
from skopt import gp_minimize
from skopt.space import Integer

# this took about an hour
search_space = list()
for i in range(5):
    search_space.append(Integer(2, 500, name=f"p{i}"))
time_start = time.time()
result = gp_minimize(obj_func_skopt, search_space, n_calls=10)
print(result)
time_end = time.time()
print(f"Time taken {time_end-time_start}")

# ok results
# fun: 0.3303020837527442
# x: [np.int64(467), np.int64(312), np.int64(377), np.int64(445), np.int64(152)]

[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.18534848 0.97636364]
[ 0.00319529  0.11207886  0.19603182 ...  0.15026209 -0.07134906
  0.08224575]
[ 0.22303895 -0.26485936  0.01530204 ...  0.01687752  0.05273895
 -0.00360871]
[ 0.0500158  -0.10737012 -0.07253413 ...  0.00027411  0.00577622
  0.0190893 ]
[-0.01213502  0.37074462  0.2513331  ... -0.09823516 -0.15988213
 -0.01213502]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 2.58634286 ... 1.86397585 1.00918155 0.98978212]
[-0.06635228  0.37629595  0.0550859  ...  0.06758971  0.04894126
 -0.06635228]
[ 0.07110089  0.28668838 -0.10511274 ...  0.14868546  0.0603098
  0.07110089]
[-0.00394595 -0.20914449  0.1265228  ...  0.00960049 -0.08302613
 -0.03768355]
[-0.03460922  0.08198381  0.02953271 ...  0.1136396   0.03067969
  0.17797456]
[0. 0. 0. ... 0. 0. 0.]
[0.90818382 1.71463158 2.58634286 ... 1.75236905 1.18534848 0.90818382]
[-0.38877119 -0.08383341  0.20505888 ...  0.06014207 -0.06998391
 -0.38877119]
[ 

In [ ]:
# Mango implementation 
from mango import Tuner, scheduler

param_space = dict()
for i in range(5):
    param_space[f'p{i}'] = range(2,500)

# Using 4 threads/cores and running in parallel -> 8 iterations total technically
config = {
    'num_iteration': 2,  
    'batch_size': 4     
}


@scheduler.parallel(n_jobs=4)
def obj_func_mango(**params):
    min_samples_split = [params['p0'],params['p1'],params['p2'],params['p3'],params['p4']]
    model = RegressionTreeEnsemble(5, min_samples_split)
    model.fit_tree(X_train, y_train)
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_true = y_test, y_pred = predictions)
    return mse

# Took about 0.5 hours (less iterations than skopt)
tuner = Tuner(param_space, obj_func_mango, config)
time_start = time.time()
results = tuner.minimize()  # Run Tuner
best_params = results["best_params"]
best_objective = results["best_objective"]
print(f'Optimal value of parameters: {best_params} and objective: {best_objective}')
time_end = time.time()
print(f"Time taken {time_end-time_start}")


# Best score: 0.34505922660458643: 100%|██████████| 2/2 [19:38<00:00, 589.42s/it]
# Optimal value of parameters: {'p0': 260, 'p1': 462, 'p2': 373, 'p3': 322, 'p4': 389} and objective: 0.34505922660458643





/Users/oliverbrown/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0. 0. 0. ... 0. 0. 0.]
[1.0574     2.11323333 2.58634286 ... 1.83387879 1.00930769 0.86240625]
[-0.12748118 -0.33901002 -0.04496224 ...  0.01423335 -0.1319647
 -0.04408779]
[ 0.0023938   1.62340671 -0.0804055  ...  0.14534637 -0.18999075
  0.08476494]
[ 0.19535155 -0.08742159  0.07600418 ...  0.0265682   0.02857726
 -0.22744552]
[-0.15285064  0.32538977 -0.02273998 ... -0.09636065  0.00538192
 -0.15285064]
[0. 0. 0. ... 0. 0. 0.]
[1.27242857 2.11323333 3.17782609 ... 1.83387879 1.00930769 0.86240625]
[ 0.00937171  0.1105013  -0.11309459 ...  0.02869235 -0.20712765
  0.00937171]
[ 0.08167854 -0.07288725  0.00525214 ...  0.00789769  0.13227394
  0.08167854]
[ 0.11167932 -0.4643877  -0.1361267  ...  0.08284384 -0.11082298
 -0.1597595 ]
[-0.06661407  0.4420307   0.01819847 ...  0.02261816 -0.01004165
  0.02470674]


  0%|          | 0/2 [00:00<?, ?it/s]

[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[ 0.02631309  0.08285828  0.41634951 ... -0.28697799  0.14159147
  0.02631309]
[-0.09096693  0.25225772  0.05164206 ...  0.09321147 -0.10408965
  0.09589317]
[-0.17032554  0.32813618  0.1114164  ... -0.10409002  0.00795214
 -0.17032554]
[ 0.08850054 -0.37228057  0.32772658 ... -0.03453577  0.04366228
  0.60037485]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 2.58634286 ... 1.86397585 1.18534848 0.98978212]
[-0.0746065   0.33735427  0.05660657 ... -0.04013535 -0.07438199
 -0.00736135]
[-0.05381285  0.03922477  0.15324455 ... -0.0165271  -0.11882266
 -0.05381285]
[0.13123566 0.07939093 0.09986667 ... 0.12429424 0.02509501 0.13123566]
[ 0.05425598  0.04206933  0.04235805 ... -0.04022182  0.11784325
 -0.07672818]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[-0.27915279  0.32351365  0.41338376 ... -0.29192433  0.0836789
 -0.27915279]
[ 0.08126

Best score: 0.34505922660458643:  50%|█████     | 1/2 [08:08<08:08, 488.52s/it]

[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 2.58634286 ... 1.86397585 1.18534848 0.98978212]
[-0.26963429  0.37258918  0.18770012 ... -0.00170469 -0.11423576
 -0.26963429]
[ 0.09704157 -0.04722873  0.30004708 ...  0.03441733 -0.03854533
  0.0771029 ]
[ 0.00034336 -0.02866799 -0.1342181  ...  0.23342681 -0.02792986
  0.07789917]
[-0.15482016  0.04436585  0.03993954 ... -0.1049585  -0.08105145
  0.04436585]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 2.11323333 2.58634286 ... 1.75236905 1.00930769 0.97636364]
[-0.10646542  0.16225     0.05172353 ... -0.00276896 -0.04175478
 -0.23482536]
[ 0.02222249 -0.07731158 -0.27063673 ...  0.1161701   0.16104527
  0.02222249]
[ 0.0028037   0.01201037  0.08214921 ...  0.03816071 -0.06219864
 -0.03236702]
[ 0.15141331 -0.25314721  0.23411783 ...  0.06139926 -0.0673281
  0.09919103]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 2.11323333 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[-0.32480891  0.76495383  0.09902037 ...  0.14053113 -0.01193254
 -0.32480891]
[ 

Best score: 0.34505922660458643: 100%|██████████| 2/2 [19:38<00:00, 589.42s/it]

Optimal value of parameters: {'p0': 260, 'p1': 462, 'p2': 373, 'p3': 322, 'p4': 389} and objective: 0.34505922660458643
Time taken 1673.7737519741058


### Another Approach

To try get below 0.5 mse, we opted for finding the hyperparameters for each tree in the Ensemble one at a time instead of all at once. We alter our objective function to instead fit an Ensemble Tree with 5 trees, to fit an Ensemble with len(min_samples_split). And then we tell our optimiser to optimise on 1 tree first, finding the optimal min_samples_split $m_0$ for $T_0$, and then we tell our optimiser to optimise on 2 trees, where $m_0$ is already given and the optimiser must find $m_1$, so at each $T_i$ we are trying to find $m_i$ only and we are given all $m_0$ to $m_{i-1}$ already.


In [ ]:
# new objective function
from mango import Tuner, scheduler

@scheduler.parallel(n_jobs=4)
def obj_func_mango_two(**params):
    min_samples_split =  []
    for i in range(len(params)):
        min_samples_split.append(params[f'p{i}'])
    model = RegressionTreeEnsemble(len(min_samples_split), min_samples_split)
    model.fit_tree(X_train, y_train)
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_true = y_test, y_pred = predictions)
    return mse

current_optimal_params = []
optimal_params_mse = []

time_start = time.time()
# iterating over each tree we want to find min_samples_split value for
for n in range(1,6):
    print(f"finding minimizer for tree {n}")
    print(current_optimal_params)
    print(optimal_params_mse)
    param_space = dict()
    # define the parameter space with values ranging from 2 to 500
    for i in range(n):
        param_space[f'p{i}'] = range(2,500)
    # then, if we have already found optimal parameters for some trees we replace them in the parameter space with just a singular value
    for i in range(len(current_optimal_params)):
        param_space[f'p{i}'] = range(current_optimal_params[i], current_optimal_params[i]+1)

    # This time 4 cores run in parallel and 20 calls to the objective function are made
    config = {
        'num_iteration': 2,  
        'batch_size': 4 
    }

    tuner = Tuner(param_space, obj_func_mango_two, config)
    results = tuner.minimize()  # Run Tuner
    # we store the best mss in an array, and the mse for each mss
    best_params = results["best_params"]
    current_optimal_params.append(best_params[f"p{n-1}"])
    optimal_params_mse.append(results["best_objective"])


print(f"Optimal min_samples_split array: {current_optimal_params}")
print(f"The best mse for each tree: {optimal_params_mse}")
time_end = time.time()
print(f"Time taken {time_end-time_start}")

# 75 mins
# Best score: 0.36528084045144876: 100%|██████████| 2/2 [15:56<00:00, 478.24s/it]
# Optimal min_samples_split array: [116, 242, 419, 279, 371]
# num_iteration 2, batch_size 4, n_jobs 4



finding minimizer for tree 1
[]
[]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]


  0%|          | 0/2 [00:00<?, ?it/s]

[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 2.58634286 ... 1.86397585 1.00918155 0.98978212]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 2.58634286 ... 1.86397585 1.18534848 0.98978212]
[0. 0. 0. ... 0. 0. 0.]
[1.27242857 2.11323333 3.17782609 ... 1.83387879 1.00930769 0.86240625]


Best score: 0.36555691265322615:  50%|█████     | 1/2 [00:46<00:46, 46.13s/it]

[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 2.58634286 ... 1.86397585 1.00918155 0.98978212]
[0. 0. 0. ... 0. 0. 0.]
[0.93895918 1.71463158 2.58634286 ... 1.75236905 1.18534848 0.93895918]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]


...

[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[-0.29358103  0.10997682  0.14921936 ...  0.04768359 -0.04896811
  0.06721281]
[-0.02841575  0.35661798  0.0129173  ...  0.14904206 -0.00837055
 -0.02841575]
[ 0.0866767  -0.0741023   0.1494608  ... -0.08197448 -0.08853027
 -0.01996273]
[ 0.0334536   0.03853506 -0.10364481 ...  0.06678096  0.0117054
  0.0334536 ]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[-0.29358103  0.10997682  0.14921936 ...  0.04768359 -0.04896811
  0.06721281]
[-0.02841575  0.35661798  0.0129173  ...  0.14904206 -0.00837055
 -0.02841575]
[ 0.0866767  -0.0741023   0.1494608  ... -0.08197448 -0.08853027
 -0.01996273]
[ 0.04022054  0.03853506 -0.10364481 ...  0.06678096  0.0117054
  0.04022054]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.1050434

Best score: 0.36528084045144876:  50%|█████     | 1/2 [08:02<08:02, 482.37s/it]

[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[-0.29358103  0.10997682  0.14921936 ...  0.04768359 -0.04896811
  0.06721281]
[-0.02841575  0.35661798  0.0129173  ...  0.14904206 -0.00837055
 -0.02841575]
[ 0.0866767  -0.0741023   0.1494608  ... -0.08197448 -0.08853027
 -0.01996273]
[ 0.05878982  0.03853506 -0.10364481 ...  0.06678096  0.0117054
  0.05878982]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[0.97636364 1.71463158 2.58634286 ... 1.75236905 1.10504348 0.97636364]
[-0.29358103  0.10997682  0.14921936 ...  0.04768359 -0.04896811
  0.06721281]
[-0.29358103  0.10997682  0.14921936 ...  0.04768359 -0.04896811
  0.06721281]
[-0.02841575  0.35661798  0.0129173  ...  0.14904206 -0.00837055
 -0.02841575]
[-0.02841575  0.35661798  0.0129173  ...  0.14904206 -0.00837055
 -0.02841575]
[ 0.0866767  -0.0741023   0.1494608  ... -0.08197448 -0.08853027
 -0.01996273]
[ 

Best score: 0.36528084045144876: 100%|██████████| 2/2 [15:56<00:00, 478.24s/it]

Optimal min_samples_split array: [116, 242, 419, 279, 371]
The best mse for each tree: [np.float64(0.36555691265322615), np.float64(0.3486166679823877), np.float64(0.3573467048066693), np.float64(0.35710751617293346), np.float64(0.36528084045144876)]
Time taken 4443.088612794876


### Results

Our best performing optimiser was Skopt, which found an MSE of $0.3303$ when evaluated on test data.
The min_samples_split array is $[467,312,377,445,152]$, we can see that the training data is very large and prefers higher mss values.


In [ ]:

min_samples = [467,312,377,445,152]
rte = RegressionTreeEnsemble(5, min_samples)
rte.fit_tree(X_train, y_train)
rte_predictions = rte.predict(X_test)
print(f"MSE produced by an Ensemble Regression Tree with best parameters found: {mean_squared_error(y_test, rte_predictions)}")

[0. 0. 0. ... 0. 0. 0.]
[0.98978212 1.55701765 1.72766144 ... 2.03436601 1.00918155 0.98978212]
[-0.29211524  0.30342622  0.41004679 ... -0.28695106  0.08606622
 -0.29211524]
[ 0.12064313  0.03930225  0.0727679  ... -0.12905928 -0.08789292
  0.1638241 ]
[-0.03864698 -0.15210551  0.0036708  ...  0.13846384  0.06228236
 -0.03864698]
[ 0.05172487 -0.10117822  0.17782535 ...  0.08459111 -0.01631989
  0.12929237]
MSE produced by an Ensemble Regression Tree with best parameters found: 0.3303020837527442


In [154]:
# In comparison with a single tree
rte = RegressionTree(467)
rte.fit_tree(X_train, y_train)
rte_predictions = rte.predict(X_test)
print(f"MSE produced by a Regression Tree with best parameter found: {mean_squared_error(y_test, rte_predictions)}")


MSE produced by a Regression Tree with best parameter found: 0.44849568967978215
